In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import explained_variance_score, \
    mean_absolute_error, \
    median_absolute_error
from sklearn.model_selection import train_test_split

In [2]:
# read in the csv data into a pandas data frame and set the date as the index
df = pd.read_csv('los_angeles_grouped_dates.csv')

# execute the describe() function and transpose the output so that it doesn't overflow the width of the screen
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,100.0,696.305915,243.560441,103.466667,543.807994,788.786866,913.078125,967.750000
LATITUDE,100.0,34.092589,0.010130,34.070194,34.085612,34.090528,34.097040,34.133871
LONGITUDE,100.0,-118.264067,0.014761,-118.299943,-118.276855,-118.264438,-118.253003,-118.228762
ELEVATION,100.0,384.799970,24.680136,322.707143,362.192529,386.148653,402.892500,455.148148
EMNT,100.0,3.369970,1.888607,-1.706250,1.991774,3.101786,4.374266,8.111111
EMXT,100.0,28.143166,2.821956,19.582143,25.924934,28.089063,30.154924,35.060714
TAVG,100.0,14.210740,1.986627,10.042857,12.863382,13.922350,15.619657,18.858621


In [3]:
# execute the info() function
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  100 non-null    float64
 1   LATITUDE    100 non-null    float64
 2   LONGITUDE   100 non-null    float64
 3   ELEVATION   100 non-null    float64
 4   EMNT        100 non-null    float64
 5   EMXT        100 non-null    float64
 6   TAVG        100 non-null    float64
dtypes: float64(7)
memory usage: 5.6 KB


In [4]:
# X will be a pandas dataframe of all columns except meantempm
X = df[[col for col in df.columns if col != 'TAVG']]

# y will be a pandas series of the meantempm
y = df['EMNT']

In [5]:
# split data into training set and a temporary set using sklearn.model_selection.traing_test_split
X_train, X_tmp, y_train, y_tmp = train_test_split(X, y, test_size=0.2, random_state=23)

In [6]:
# take the remaining 20% of data in X_tmp, y_tmp and split them evenly
X_test, X_val, y_test, y_val = train_test_split(X_tmp, y_tmp, test_size=0.5, random_state=23)

X_train.shape, X_test.shape, X_val.shape
print("Training instances   {}, Training features   {}".format(X_train.shape[0], X_train.shape[1]))
print("Validation instances {}, Validation features {}".format(X_val.shape[0], X_val.shape[1]))
print("Testing instances    {}, Testing features    {}".format(X_test.shape[0], X_test.shape[1]))


Training instances   80, Training features   6
Validation instances 10, Validation features 6
Testing instances    10, Testing features    6


In [7]:
feature_cols = [tf.feature_column.numeric_column(col) for col in X.columns]

In [8]:
regressor = tf.estimator.DNNRegressor(feature_columns=feature_cols,
                                      hidden_units=[50, 50],
                                      model_dir='tf_wx_model')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'tf_wx_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff364c616a0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [9]:
def wx_input_fn(X, y=None, num_epochs=None, shuffle=True, batch_size=400):
    return tf.estimator.inputs.pandas_input_fn(x=X,
                                               y=y,
                                               num_epochs=num_epochs,
                                               shuffle=shuffle,
                                               batch_size=batch_size)

In [10]:
evaluations = []
STEPS = 400
for i in range(100):
    regressor.train(input_fn=wx_input_fn(X_train, y=y_train), steps=STEPS)
    evaluations.append(regressor.evaluate(input_fn=wx_input_fn(X_val,
                                                               y_val,
                                                               num_epochs=1,
                                                               shuffle=False)))

AttributeError: module 'tensorflow_core.estimator' has no attribute 'inputs'